In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 50)

from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX, VARMAXResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error,mean_absolute_error
from pmdarima import auto_arima
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [2]:
Grab = pd.read_csv('GrabAssignment.csv')

In [3]:
Grab.head()

,geohash6,day,timestamp,demand
0,qp03wc,18,20:0,0.020072
1,qp03pn,10,14:30,0.024721
2,qp09sw,9,6:15,0.102821
3,qp0991,32,5:0,0.088755
4,qp090q,15,4:0,0.074468


In [4]:
Grab.shape

(4206321, 4)

In [5]:
Grab.columns

Index(['geohash6', 'day', 'timestamp', 'demand'], dtype='object')

In [6]:
Grab.dtypes

geohash6      object
day            int64
timestamp     object
demand       float64
dtype: object

In [7]:
# Each day should have 96 timestamp (15 min), here to check out which day didnt have complete timestamp cycle
a_list = []
for c in range(1,61):
    lens = len(Grab[Grab["day"] == c].timestamp.value_counts())
    if lens != 96:
        a_list.append(c)
print(a_list)

[18]


In [8]:
#Regroup data to make the day and timestamp in order and unstack geohash
GrabFull = Grab.groupby(['geohash6', 'day', 'timestamp'])['demand'].mean().unstack('geohash6')

In [9]:
GrabFull.columns

Index(['qp02yc', 'qp02yf', 'qp02yu', 'qp02yv', 'qp02yy', 'qp02yz', 'qp02z1',
       'qp02z3', 'qp02z4', 'qp02z5',
       ...
       'qp0djv', 'qp0djw', 'qp0djy', 'qp0dn0', 'qp0dn1', 'qp0dn4', 'qp0dn5',
       'qp0dnh', 'qp0dnj', 'qp0dnn'],
      dtype='object', name='geohash6', length=1329)

In [10]:
# Creating the DataFrame to fillup the NaN value in Raw Data to complete timestamp cycle for timeseries forecasting
fillup_ts = pd.DataFrame(GrabFull.columns)
fillup_ts["day"] = 18
fillup_ts["demand"] = np.NaN

In [11]:
fillup_ts

,geohash6,day,demand
0,qp02yc,18,NaN
1,qp02yf,18,NaN
2,qp02yu,18,NaN
3,qp02yv,18,NaN
4,qp02yy,18,NaN
5,qp02yz,18,NaN
6,qp02z1,18,NaN
7,qp02z3,18,NaN
8,qp02z4,18,NaN
9,qp02z5,18,NaN


In [12]:
fillup_ts_str = pd.Series([str("9:45"),str("10:0"),str("10:15"),str("12:45"),str("11:30"),str("11:45"),str("12:0"),str("12:15"),str("12:30")])

In [13]:
fillup_ts_full = pd.DataFrame()
for ts in fillup_ts_str:
    fillup_ts["timestamp"] = ts
    fillup_ts_full = pd.concat([fillup_ts_full,fillup_ts], ignore_index=True)
    

In [14]:
fillup_ts_full

,geohash6,day,demand,timestamp
0,qp02yc,18,NaN,9:45
1,qp02yf,18,NaN,9:45
2,qp02yu,18,NaN,9:45
3,qp02yv,18,NaN,9:45
4,qp02yy,18,NaN,9:45
5,qp02yz,18,NaN,9:45
6,qp02z1,18,NaN,9:45
7,qp02z3,18,NaN,9:45
8,qp02z4,18,NaN,9:45
9,qp02z5,18,NaN,9:45


In [15]:
Grab_done = pd.concat([Grab,fillup_ts_full], ignore_index=True, sort=False)

In [16]:
Grab_done

,geohash6,day,timestamp,demand
0,qp03wc,18,20:0,0.020072
1,qp03pn,10,14:30,0.024721
2,qp09sw,9,6:15,0.102821
3,qp0991,32,5:0,0.088755
4,qp090q,15,4:0,0.074468
5,qp03tu,1,12:15,0.023843
6,qp096d,25,3:30,0.007460
7,qp03nr,51,20:45,0.000293
8,qp093r,48,6:15,0.054170
9,qp03r2,4,22:15,0.123463


In [17]:
Grab_done['timestamp'] = pd.to_datetime(Grab_done['timestamp'], format='%H:%M').dt.time

In [18]:
grab_train = Grab_done.groupby(['geohash6', 'day', 'timestamp'])['demand'].mean().unstack('geohash6')

In [19]:
grab_train.shape

(5856, 1329)

In [20]:
grab_train.isnull().sum().sum()

3576303

In [21]:
#Merge the consecutive day and timestamp into one time series with a assigned datetime and 15min frequency
grab_train = grab_train.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5856, freq='15min'))

In [22]:
grab_train = grab_train.fillna(0)

In [23]:
grab_train.head(10)

geohash6,qp02yc,qp02yf,qp02yu,qp02yv,qp02yy,qp02yz,qp02z1,qp02z3,qp02z4,qp02z5,qp02z6,qp02z7,qp02z9,qp02zc,qp02zd,qp02ze,qp02zf,qp02zg,qp02zh,qp02zj,qp02zk,qp02zm,qp02zn,qp02zp,qp02zq,...,qp0djb,qp0djc,qp0djd,qp0dje,qp0djf,qp0djg,qp0djh,qp0djj,qp0djk,qp0djm,qp0djn,qp0djq,qp0djs,qp0djt,qp0dju,qp0djv,qp0djw,qp0djy,qp0dn0,qp0dn1,qp0dn4,qp0dn5,qp0dnh,qp0dnj,qp0dnn
2019-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.022396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.009482,0.0,0.000000,0.000000,0.003641,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0006,0.0,0.000000,0.000000,0.038979,0.000000,0.010554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.012865,0.000000,0.000000,0.0,0.001304,0.000000,0.002454,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.004056,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.022022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.004910,0.000000,0.000000,0.0,0.020167,0.000000,0.000000,0.000000,0.0,0.000503,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.009381,0.0
2019-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.070525,0.001295,0.000000,0.000000,0.008541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.001248,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.083947,0.000000,0.045482,0.047912,0.006503,0.000000,0.000000,0.000000,0.049005,0.047575,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.009687,0.000000,0.051739,0.0,0.010428,0.000000,0.000000,0.0,0.0,0.0,0.008253,0.0,0.0,0.0,0.0,0.0,0.002701,0.0
2019-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.045083,0.000000,0.000000,0.031414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060752,0.000000,0.000000,...,0.0,0.0,0.011394,0.000000,0.000000,0.0,0.009599,0.017358,0.000000,0.000000,0.0,0.020719,0.000000,0.000000,0.0,0.0,0.0,0.006971,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.099317,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.048360,0.000954,0.000000,0.000000,0.000000,0.000000,0.000000,0.104351,0.075672,0.000630,0.066387,...,0.0,0.0,0.000000,0.000471,0.012844,0.0,0.027026,0.029316,0.007121,0.005744,0.0,0.000000,0.000993,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.019563,0.0
2019-01-01 01:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.011855,0.003631,0.000000,0.000000,0.039394,0.000000,0.000000,0.000000,0.000000,0.019048,0.036850,0.079985,0.055821,0.071413,...,0.0,0.0,0.000000,0.000000,0.003883,0.0,0.004794,0.000000,0.019596,0.031227,0.0,0.023070,0.000721,0.000000,0.0,0.0,0.0,0.001858,0.0,0.0,0.0,0.0,0.0,0.007193,0.0
2019-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.029813,0.0,0.0,0.0000,0.0,0.000000,0.045931,0.000000,0.000000,0.000000,0.000000,0.023930,0.000000,0.082367,0.000000,0.019131,0.022062,0.036493,0.012899,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.016685,0.024294,0.034116,0.018435,0.0,0.011025,0.005066,0.011381,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.099991,0.0,0.0,0.0000,0.0,0.019614,0.019863,0.000000,0.016487,0.000000,0.000000,0.044045,0.014075,0.237685,0.000000,0.042676,0.124104,0.038605,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000553,0.006702,0.032528,0.007979,0.0,0.000000,0.019812,0.002717,0.0,0.0,0.0,0.008936,0.0,0.0,0.0,0.0,0.0,0.007224,0.0


# Dimensionality Reduction, PCA to 5D with retained 80% variance

In [24]:
from sklearn.decomposition import PCA

In [25]:
pca = PCA(n_components=5)

In [26]:
train_2D = pca.fit_transform(grab_train[0:-672])

In [27]:
pca.explained_variance_ratio_.sum()

0.8085070179723179

In [28]:
pca.explained_variance_ratio_

array([0.65180352, 0.06130845, 0.04398342, 0.02706075, 0.02435087])

In [29]:
train_2D = pd.DataFrame(data = train_2D, columns=["z1","z2","z3","z4","z5"])

In [30]:
train_2D = train_2D.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5184, freq='15min'))

In [31]:
train_2D.shape

(5184, 5)

# 1. Modelling VAR(96, trend="ctt"), Train(5184), Test(672), PCA > 5D, RMSE = 0.04694127

In [45]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX, VARMAXResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error,mean_absolute_error
from pmdarima import auto_arima
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split

In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
for c in train_2D.columns: # All z1-z5 data stationary
    adf_test(train_2D[c],title=c)

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF = train_2D[:-nobs]
testF = grab_train[-nobs:]

In [ ]:
model_VAR = VAR(trainF)

In [ ]:
aic_dict = {}
for i in range(94,97):
    results = model_VAR.fit(i)
    aic_dict[i] = results.aic

In [ ]:
model_VAR.endog_names

In [ ]:
# To return the key_values, where it value are smallest
min(aic_dict, key=aic_dict.get)

In [ ]:
# To return the max & min value in a dictionary object
key_max = max(aic_dict.keys(), key=(lambda k: aic_dict[k]))
key_min = min(aic_dict.keys(), key=(lambda k: aic_dict[k]))
print('Maximum Value: ',aic_dict[key_max])
print('Minimum Value: ',aic_dict[key_min])

In [ ]:
results_VAR = model_VAR.fit(maxlags=96, trend="ctt") 
results_VAR.summary()

In [ ]:
results_VAR.k_ar

In [ ]:
results_VAR.aic

In [ ]:
lagged_values_VAR = trainF.values[-96:]

In [ ]:
z = results_VAR.forecast(y=lagged_values_VAR, steps=672) 
df_forecast_VAR = pca.inverse_transform(z)

In [ ]:
#Set the time frame to be same as test dataset
idxH = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR = pd.DataFrame(df_forecast_VAR, index=idxH, columns=testF.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR[df_forecast_VAR < 0] = 0
df_forecast_VAR[df_forecast_VAR > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF,df_forecast_VAR)))
print(mean_absolute_error(testF,df_forecast_VAR)*100)
print(testF.mean().sum())

In [ ]:
#The model residual mean
#Residual mean close to zero which is a good indication for the forecasting
np.mean(results_VAR.resid).sum()

In [ ]:
#Visualise the forecast values and dataset values for all of the geohash6
df_forecast_VAR.iplot()
testF.iplot()

# 2. Modelling VAR(96, trend="ctt"), Train(-3360:-672), Test(672), PCA > 5D, RMSE = 0.05020833

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainHalf = train_2D[-3360:-nobs]
testHalf = grab_train[-nobs:]

In [ ]:
model_VAR_H = VAR(trainHalf)

In [ ]:
results_VAR_H = model_VAR_H.fit(maxlags=96, trend="ctt") 
results_VAR_H.summary()

In [ ]:
results_VAR_H.k_ar

In [ ]:
results_VAR_H.aic

In [ ]:
lagged_values_VAR_H = trainHalf.values[-96:]

In [ ]:
z_H = results_VAR_H.forecast(y=lagged_values_VAR_H, steps=672) 
df_forecast_VAR_H = pca.inverse_transform(z_H)

In [ ]:
#Set the time frame to be same as test dataset
idxH_H = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_H = pd.DataFrame(df_forecast_VAR_H, index=idxH_H, columns=testHalf.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_H[df_forecast_VAR_H < 0] = 0
df_forecast_VAR_H[df_forecast_VAR_H > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testHalf,df_forecast_VAR_H)))
print(mean_absolute_error(testHalf,df_forecast_VAR_H)*100)
print(testHalf.mean().sum())

In [ ]:
#Visualise the forecast values and dataset values for all of the geohash6
df_forecast_VAR_H.iplot()
testHalf.iplot()

# 3. Modelling VAR(96, trend="ctt",exog= Weekday), Train(5184), Test(672), PCA > 5D, RMSE = 0.04711870

In [33]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = train_2D[:-nobs]
testF_Exo = grab_train[-nobs:]

In [34]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [35]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [36]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [37]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [38]:
trainF_Exo_feature.columns

Index(['z1', 'z2', 'z3', 'z4', 'z5', 'Weekday', 'Weekday_Friday',
       'Weekday_Monday', 'Weekday_Saturday', 'Weekday_Sunday',
       'Weekday_Thursday', 'Weekday_Tuesday', 'Weekday_Wednesday'],
      dtype='object')

In [39]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [40]:
trainF_Exo_feature.sample(5)

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
2019-02-11 06:30:00,0,1,0,0,0,0,0
2019-01-09 00:30:00,0,0,0,0,0,0,1
2019-01-07 07:45:00,0,1,0,0,0,0,0
2019-02-02 07:30:00,0,0,1,0,0,0,0
2019-02-09 10:30:00,0,0,1,0,0,0,0


In [41]:
model_VAR_F_Exo = VAR(trainF_Exo, exog=trainF_Exo_feature)

In [42]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=96, trend="ctt") 
results_model_VAR_F_Exo.summary()

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/vector_ar/var_model.py:1387: RuntimeWarning:

invalid value encountered in sqrt

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning:

invalid value encountered in greater

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning:

invalid value encountered in less

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning:

invalid value encountered in less_equal



  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 31, Aug, 2019
Time:                     19:22:38
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -21.3489
Nobs:                     4416.00    HQIC:                  -23.6448
Log likelihood:           26089.7    FPE:                1.54844e-11
AIC:                     -24.8958    Det(Omega_mle):     9.14958e-12
--------------------------------------------------------------------
Results for equation z1
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.005682              NAN              NAN             NAN
trend           -0.000025         0.000011           -2.288           0.022
trend**2         0.000000         0.000000            2.671           0.008
exog0  

In [43]:
results_model_VAR_F_Exo.aic

-24.895778184771164

In [44]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-96:]

In [46]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672, exog_future=testF_Exo_feature) 
df_forecast_VAR_F_Exo = pca.inverse_transform(z_F_Exo)

In [47]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(df_forecast_VAR_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [48]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [49]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

0.04711869748444822
2.2579737205471724
81.90959266258383


# 4. Modelling VAR(96, trend="ctt",exog= Weekday), Train(-3360:-672), Test(672), PCA > 5D, RMSE = 0.04785709

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = train_2D[-3360:-nobs]
testF_Exo = grab_train[-nobs:]

In [ ]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [ ]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [ ]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [ ]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [ ]:
trainF_Exo_feature.columns

In [ ]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [ ]:
trainF_Exo_feature.sample(5)

In [ ]:
model_VAR_F_Exo = VAR(trainF_Exo, exog=trainF_Exo_feature)

In [ ]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=96, trend="ctt") 
results_model_VAR_F_Exo.summary()

In [ ]:
results_model_VAR_F_Exo.aic

In [ ]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-96:]

In [ ]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672, exog_future=testF_Exo_feature) 
df_forecast_VAR_F_Exo = pca.inverse_transform(z_F_Exo)

In [ ]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(df_forecast_VAR_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

# 5. Modelling VAR(7, trend="ctt"), Train(5184), Test(672), Full- Dimension, RMSE = 0.07916411

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainHalf = grab_train[0:-nobs]
testHalf = grab_train[-nobs:]

In [ ]:
np.fill_diagonal(trainHalf.values, trainHalf + 1)

In [ ]:
model_VAR_H = VAR(trainHalf)

In [ ]:
results_VAR_H = model_VAR_H.fit(maxlags=7, trend="ctt")

In [ ]:
results_VAR_H.k_ar

In [ ]:
lagged_values_VAR_H = trainHalf.values[-7:]

In [ ]:
z_H = results_VAR_H.forecast(y=lagged_values_VAR_H, steps=672)

In [ ]:
#Set the time frame to be same as test dataset
idxH_H = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_H = pd.DataFrame(z_H, index=idxH_H, columns=testHalf.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_H[df_forecast_VAR_H < 0] = 0
df_forecast_VAR_H[df_forecast_VAR_H > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testHalf,df_forecast_VAR_H)))
print(mean_absolute_error(testHalf,df_forecast_VAR_H)*100)
print(testHalf.mean().sum())

# 6. Modelling VAR(7, trend="ctt"), Train(-3360:-672), Test(672), Full- Dimension, RMSE = 0.08025244

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672
trainHalf = grab_train[-3360:-nobs]
testHalf = grab_train[-nobs:]

In [ ]:
np.fill_diagonal(trainHalf.values, trainHalf + 1)

In [ ]:
model_VAR_H = VAR(trainHalf)

In [ ]:
results_VAR_H = model_VAR_H.fit(maxlags=7, trend="ctt")

In [ ]:
results_VAR_H.k_ar

In [ ]:
lagged_values_VAR_H = trainHalf.values[-7:]

In [ ]:
z_H = results_VAR_H.forecast(y=lagged_values_VAR_H, steps=672)

In [ ]:
#Set the time frame to be same as test dataset
idxH_H = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_H = pd.DataFrame(z_H, index=idxH_H, columns=testHalf.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_H[df_forecast_VAR_H < 0] = 0
df_forecast_VAR_H[df_forecast_VAR_H > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testHalf,df_forecast_VAR_H)))
print(mean_absolute_error(testHalf,df_forecast_VAR_H)*100)
print(testHalf.mean().sum())

# 7. Modelling VAR(7, trend="ctt",exog= Weekday), Train(5184), Test(672), Full-Dimension, RMSE = 0.57290424

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = grab_train[0:-nobs]
testF_Exo = grab_train[-nobs:]

In [ ]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [ ]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [ ]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [ ]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [ ]:
trainF_Exo_feature.columns

In [ ]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [ ]:
trainF_Exo_feature.sample(5)

In [ ]:
model_VAR_F_Exo = VAR(trainF_Exo, exog=trainF_Exo_feature)

In [ ]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=7, trend="ctt") 

In [ ]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-7:]

In [ ]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672, exog_future=testF_Exo_feature) 

In [ ]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(z_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

# 8. Modelling VAR(7, trend="ctt",exog= Weekday), Train(-3360:-672), Test(672), Full-Dimension, RMSE = 0.55819723

In [ ]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = grab_train[-3360:-nobs]
testF_Exo = grab_train[-nobs:]

In [ ]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [ ]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [ ]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [ ]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [ ]:
trainF_Exo_feature.columns

In [ ]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [ ]:
trainF_Exo_feature.sample(5)

In [ ]:
model_VAR_F_Exo = VAR(trainF_Exo, exog=trainF_Exo_feature)

In [ ]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=7, trend="ctt") 

In [ ]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-7:]

In [ ]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672, exog_future=testF_Exo_feature) 

In [ ]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(z_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [ ]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [ ]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

In [ ]:
#Visualise the forecast values and dataset values for all of the geohash6
df_forecast_VAR_F_Exo.iplot()
testF_Exo.iplot()

# 9. Modelling VARMA(order = (3, 1)), Train(5184), Test(672), 5D-Dimension, RMSE = 0.07555706

In [50]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = train_2D[:-nobs]
testF_Exo = grab_train[-nobs:]

In [51]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [52]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [53]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [54]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [55]:
trainF_Exo_feature.columns

Index(['z1', 'z2', 'z3', 'z4', 'z5', 'Weekday', 'Weekday_Friday',
       'Weekday_Monday', 'Weekday_Saturday', 'Weekday_Sunday',
       'Weekday_Thursday', 'Weekday_Tuesday', 'Weekday_Wednesday'],
      dtype='object')

In [56]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [57]:
trainF_Exo_feature.sample(5)

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
2019-01-05 19:30:00,0,0,1,0,0,0,0
2019-01-29 10:00:00,0,0,0,0,0,1,0
2019-02-01 23:00:00,1,0,0,0,0,0,0
2019-01-11 16:00:00,1,0,0,0,0,0,0
2019-01-03 19:45:00,0,0,0,0,1,0,0


In [ ]:
auto_arima(trainF_Exo['z1'],seasonal=True,m = 12)

In [58]:
model_VAR_F_Exo = VARMAX(trainF_Exo, order=(3,1))

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/varmax.py:152: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.



In [59]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=3, disp=False) 
results_model_VAR_F_Exo.summary()

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Dep. Variable:,"['z1', 'z2', 'z3', 'z4', 'z5']",No. Observations:,4512
Model:,"VARMA(3,1)",Log Likelihood,21815.821
,+ intercept,AIC,-43391.643
Date:,"Sat, 31 Aug 2019",BIC,-42621.903
Time:,19:25:34,HQIC,-43120.453
Sample:,01-01-2019,,
,- 02-16-2019,,
Covariance Type:,opg,,
Ljung-Box (Q):,"698.61, 300.43, 156.80, 794.30, 303.24",Jarque-Bera (JB):,"16435938.45, 1117.93, 12945.35, 1139.39, 324.99"
Prob(Q):,"0.00, 0.00, 0.00, 0.00, 0.00",Prob(JB):,"0.00, 0.00, 0.00, 0.00, 0.00"
Heteroskedasticity (H):,"1.24, 1.32, 1.14, 1.03, 1.04",Skew:,"-4.19, 0.37, -0.27, 0.31, -0.01"


In [60]:
results_model_VAR_F_Exo.aic

-43391.64262866739

In [61]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-3:]

In [62]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672) 
df_forecast_VAR_F_Exo = pca.inverse_transform(z_F_Exo)

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:320: FutureWarning:

Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/varmax.py:152: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.



In [63]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(df_forecast_VAR_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [64]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [65]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

0.07555749375261353
3.5794621930408757
81.90959266258383


# 10. Modelling SARIMAX(order(3,0,1), seasonal_order(2,0,0,4)) with Exog, Train(5184), Test(672), 1D-Dimension, RMSE = 0.07861528

In [66]:
pca = PCA(n_components=1)

In [67]:
train_2D = pca.fit_transform(grab_train[0:-672])

In [68]:
pca.explained_variance_ratio_.sum()

0.651803520982282

In [69]:
pca.explained_variance_ratio_

array([0.65180352])

In [70]:
train_2D = pd.DataFrame(data = train_2D, columns=["z1"])

In [71]:
train_2D = train_2D.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5184, freq='15min'))

In [72]:
train_2D.shape

(5184, 1)

In [73]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = train_2D[:-nobs]
testF_Exo = grab_train[-nobs:]

In [74]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [75]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [76]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [77]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [78]:
trainF_Exo_feature.columns

Index(['z1', 'Weekday', 'Weekday_Friday', 'Weekday_Monday', 'Weekday_Saturday',
       'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday',
       'Weekday_Wednesday'],
      dtype='object')

In [79]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [80]:
trainF_Exo_feature.sample(5)

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
2019-01-03 21:15:00,0,0,0,0,1,0,0
2019-01-26 06:15:00,0,0,1,0,0,0,0
2019-02-13 23:30:00,0,0,0,0,0,0,1
2019-01-14 12:15:00,0,1,0,0,0,0,0
2019-01-02 06:00:00,0,0,0,0,0,0,1


In [ ]:
auto_arima(trainF_Exo['z1'],seasonal=True,m = 4, exogenous=trainF_Exo_feature)

In [90]:
model_VAR_F_Exo = SARIMAX(trainF_Exo['z1'],exog=trainF_Exo_feature,order=(3,0,1),seasonal_order=(2,0,0,4))
results_model_VAR_F_Exo = model_VAR_F_Exo.fit()
results_model_VAR_F_Exo.summary()

/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                                z1   No. Observations:                 4512
Model:             SARIMAX(3, 0, 1)x(2, 0, 0, 4)   Log Likelihood                1265.489
Date:                           Sat, 31 Aug 2019   AIC                          -2502.978
Time:                                   19:28:54   BIC                          -2413.175
Sample:                               01-01-2019   HQIC                         -2471.339
                                    - 02-16-2019                                         
Covariance Type:                             opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Weekday_Friday       -0.1180      0.224     -0.526      0.599      -0.557       0.321
Weekday_Monday        0.1577      0.230      0.684      0.494      -0.294       0.609
Weekday_Saturday     -0.2550      0.225     -1.133      0.257      -0.696       0.186
Weekday_Sunday       -0.1869      0.231     -0.810      0.418      -0.639       0.265
Weekday_Thursday     -0.0587      0.225     -0.261      0.794      -0.500       0.382
Weekday_Tuesday       0.0264      0.228      0.116      0.908      -0.421       0.474
Weekday_Wednesday    -0.0282      0.227     -0.125      0.901      -0.472       0.416
ar.L1                 1.4049      0.027     51.898      0.000       1.352       1.458
ar.L2                -0.2106      0.035     -5.989      0.000      -0.280      -0.142
ar.L3                -0.2157      0.009    -23.454      0.000      -0.234      -0.198
ma.L1                -0.1770      0.028     -6.413      0.000      -0.231      -0.123
ar.S.L4               0.1810      0.005     39.994      0.000       0.172       0.190
ar.S.L8               0.2643      0.005     57.932      0.000       0.255       0.273
sigma2                0.0334    9.1e-05    366.426      0.000       0.033       0.034
===================================================================================
Ljung-Box (Q):                      446.94   Jarque-Bera (JB):          13044136.12
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.21   Skew:                            -4.38
Prob(H) (two-sided):                  0.00   Kurtosis:                       266.26
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [91]:
results_model_VAR_F_Exo.aic

-2502.978198876587

In [83]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-3:]

In [92]:
z_F_Exo = results_model_VAR_F_Exo.forecast(steps=672, exog=testF_Exo_feature) 


/Users/cheechingeng/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:320: FutureWarning:

Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.



In [93]:
z_F_Exo = pd.DataFrame(z_F_Exo)

In [94]:
df_forecast_VAR_F_Exo = pca.inverse_transform(z_F_Exo)

In [95]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(df_forecast_VAR_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [96]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [97]:
#The RMSE for the forecasted values and test dataset
print(np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo)))
print(mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100)
print(testF_Exo.mean().sum())

0.07871060181732027
3.7267366878644252
81.90959266258383


# 11. Dimensionality Reduction, PCA with retained 95% variance, Modelling VAR(96, trend="ctt",exog= Weekday), Train(5184), Test(672), RMSE = 0.04711870

In [24]:
from sklearn.decomposition import PCA

In [25]:
pca = PCA(0.95)

In [26]:
train_2D = pca.fit_transform(grab_train[:-672])

In [27]:
pca.explained_variance_ratio_.sum()

0.9501405036970398

In [28]:
pca.explained_variance_ratio_

array([6.51803521e-01, 6.13084516e-02, 4.39834237e-02, 2.70607501e-02,
       2.43508716e-02, 1.75100875e-02, 1.29762158e-02, 8.40748785e-03,
       6.61011213e-03, 6.24463943e-03, 4.81676383e-03, 4.72410334e-03,
       3.87567708e-03, 3.64940584e-03, 3.12047035e-03, 2.69894027e-03,
       2.48680843e-03, 2.22087156e-03, 2.08501822e-03, 2.01848902e-03,
       1.88598121e-03, 1.63494691e-03, 1.52169704e-03, 1.50807216e-03,
       1.41445432e-03, 1.32655326e-03, 1.28530012e-03, 1.20351730e-03,
       1.16560559e-03, 1.12709808e-03, 1.10392382e-03, 1.06164220e-03,
       9.99131901e-04, 9.80129237e-04, 9.58076150e-04, 9.36822241e-04,
       8.99536398e-04, 8.77785092e-04, 8.50523352e-04, 8.15951215e-04,
       7.82523108e-04, 7.78746326e-04, 7.59420882e-04, 7.41447503e-04,
       7.20550428e-04, 7.11871780e-04, 6.90015780e-04, 6.76431463e-04,
       6.67984519e-04, 6.57261678e-04, 6.38505613e-04, 6.21709019e-04,
       6.09059280e-04, 5.98429760e-04, 5.90245775e-04, 5.80509339e-04,
      

In [29]:
pca.explained_variance_ratio_.shape

(117,)

In [32]:
train_2D = pd.DataFrame(train_2D)

In [33]:
train_2D = train_2D.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5184, freq='15min'))

In [34]:
train_2D.shape

(5184, 117)

In [36]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
trainF_Exo = train_2D
testF_Exo = grab_train[-nobs:]

In [37]:
trainF_Exo_feature = trainF_Exo.copy()
testF_Exo_feature = testF_Exo.copy()

In [38]:
trainF_Exo_feature['Weekday'] = trainF_Exo_feature.index.weekday_name
testF_Exo_feature['Weekday'] = testF_Exo_feature.index.weekday_name

In [39]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [40]:
trainF_Exo_feature = create_dummies(trainF_Exo_feature,'Weekday')
testF_Exo_feature = create_dummies(testF_Exo_feature,'Weekday')

In [42]:
testF_Exo_feature.columns

Index(['qp02yc', 'qp02yf', 'qp02yu', 'qp02yv', 'qp02yy', 'qp02yz', 'qp02z1',
       'qp02z3', 'qp02z4', 'qp02z5',
       ...
       'qp0dnj', 'qp0dnn', 'Weekday', 'Weekday_Friday', 'Weekday_Monday',
       'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday',
       'Weekday_Tuesday', 'Weekday_Wednesday'],
      dtype='object', length=1337)

In [43]:
trainF_Exo_feature = trainF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
testF_Exo_feature = testF_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [44]:
trainF_Exo_feature.sample(5)

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
2019-01-22 19:45:00,0,0,0,0,0,1,0
2019-01-12 15:15:00,0,0,1,0,0,0,0
2019-02-06 22:45:00,0,0,0,0,0,0,1
2019-01-29 14:45:00,0,0,0,0,0,1,0
2019-01-15 13:00:00,0,0,0,0,0,1,0


In [50]:
model_VAR_F_Exo = VAR(trainF_Exo, exog=trainF_Exo_feature)

In [60]:
results_model_VAR_F_Exo = model_VAR_F_Exo.fit(maxlags=672, trend="ctt") 

In [61]:
results_model_VAR_F_Exo.aic

-2672.2772384458553

In [62]:
lagged_values_VAR_F_Exo = trainF_Exo.values[-672:]

In [63]:
z_F_Exo = results_model_VAR_F_Exo.forecast(y=lagged_values_VAR_F_Exo, steps=672, exog_future=testF_Exo_feature) 
df_forecast_VAR_F_Exo = pca.inverse_transform(z_F_Exo)

In [64]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_F_Exo = pd.DataFrame(df_forecast_VAR_F_Exo, index=idxH_F_Exo, columns=testF_Exo.columns)

In [65]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo < 0] = 0
df_forecast_VAR_F_Exo[df_forecast_VAR_F_Exo > 1] = 1

In [82]:
#The RMSE for the forecasted values and test dataset
RMSE = np.sqrt(mean_squared_error(testF_Exo,df_forecast_VAR_F_Exo))
MAE = mean_absolute_error(testF_Exo,df_forecast_VAR_F_Exo)*100
Test_dataset_mean = testF_Exo.mean().sum()
RMSE_divide_Test_mean = (RMSE / Test_dataset_mean) * 100
relative_error_in_100_percentage = 100 - RMSE_divide_Test_mean
print(RMSE)
print(MAE)
print(Test_dataset_mean)
print(RMSE_divide_Test_mean)
print(relative_error_in_100_percentage)

0.041840895707627154
2.1388077664917358
81.90959266258383
0.05108180171275593
99.94891819828725
